In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras_tuner import RandomSearch
train_dir = r'E:\AI\dataset_skeleton_sep\side\BicycleCrunch\training'
val_dir = r'E:\AI\dataset_skeleton_sep\side\BicycleCrunch\validation'
test_dir = r'E:\AI\dataset_skeleton_sep\side\BicycleCrunch\test'

# ImageDataGenerator 초기화
datagen = ImageDataGenerator(rescale=1./255)  # 이미지를 0과 1 사이의 값으로 정규화

# 훈련, 검증, 테스트 데이터셋을 위한 제너레이터 생성
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    shuffle=True)

validation_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    shuffle=False)

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    shuffle=False)


Found 7775 images belonging to 8 classes.
Found 1670 images belonging to 8 classes.
Found 1667 images belonging to 8 classes.


In [2]:
def build_model(hp):
    base_model = tf.keras.applications.InceptionV3(weights="imagenet",
                                                    include_top=False, 
                                                    input_shape=(128, 128, 3))
    for layer in base_model.layers:
        base_model.trainable = False
    for layer in base_model.layers[-9:]:
        base_model.trainable = True

    model = models.Sequential()
    model.add(base_model)
    model.add(layers.Flatten())
    # Dense 레이어의 유닛 수를 하이퍼파라미터로 사용
    model.add(layers.Dense(units=hp.Int('units', min_value=128, max_value=512, step=128), activation='relu'))
    model.add(layers.Dense(8, activation='softmax'))
    
    # learning_rate를 하이퍼파라미터로 사용
    model.compile(optimizer=optimizers.Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-3, sampling='LOG')),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model


In [3]:
import keras_tuner as kt

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,  # 시도할 하이퍼파라미터 조합의 최대 개수
    executions_per_trial=1,  # 각 하이퍼파라미터 설정을 평가하기 위해 모델을 훈련시킬 횟수
    directory='random_search',  # 튜닝 세션의 결과를 저장할 디렉토리 이름
    project_name='GN_BC_RS_01', # 프로젝트 이름    
)
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 128, 'max_value': 512, 'step': 128, 'sampling': 'linear'}
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.001, 'step': None, 'sampling': 'log'}


In [4]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

tuner.search(train_generator,
             steps_per_epoch=train_generator.samples // train_generator.batch_size,
             validation_data=validation_generator, 
             validation_steps=validation_generator.samples // validation_generator.batch_size,
             epochs=25,
             callbacks=[early_stopping])


# 최적의 하이퍼파라미터 가져오기
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# 최적의 하이퍼파라미터로 모델 빌드
model = tuner.hypermodel.build(best_hps)

Trial 5 Complete [00h 07m 49s]
val_accuracy: 0.625

Best val_accuracy So Far: 0.625
Total elapsed time: 00h 24m 15s


In [5]:
# 모든 최적 하이퍼파라미터 출력
for hp in best_hps.values:
    print(f"The optimal value for {hp} is {best_hps.get(hp)}")
    
# 모델 훈련
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples // train_generator.batch_size,
                    validation_data=validation_generator, 
                    validation_steps=validation_generator.samples // validation_generator.batch_size,
                    epochs=25,
                    callbacks=[early_stopping])

# 모델 평가 (테스트 데이터셋)
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print('\n테스트 정확도:', test_acc)

The optimal value for units is 512
The optimal value for learning_rate is 0.0007297440579587305
Epoch 1/25
242/242 [==============================] - 22s 76ms/step - loss: 2.1302 - accuracy: 0.1601 - val_loss: 2.0295 - val_accuracy: 0.1995
Epoch 2/25
242/242 [==============================] - 17s 72ms/step - loss: 1.8869 - accuracy: 0.2405 - val_loss: 1.8816 - val_accuracy: 0.2476
Epoch 3/25
242/242 [==============================] - 17s 72ms/step - loss: 1.7531 - accuracy: 0.2893 - val_loss: 1.8448 - val_accuracy: 0.2650
Epoch 4/25
242/242 [==============================] - 17s 72ms/step - loss: 1.6713 - accuracy: 0.3243 - val_loss: 1.7727 - val_accuracy: 0.2939
Epoch 5/25
242/242 [==============================] - 17s 72ms/step - loss: 1.4815 - accuracy: 0.3904 - val_loss: 1.6990 - val_accuracy: 0.3167
Epoch 6/25
242/242 [==============================] - 18s 73ms/step - loss: 1.3497 - accuracy: 0.4443 - val_loss: 2.3269 - val_accuracy: 0.2758
Epoch 7/25
242/242 [====================